In [1]:
import pandas as pd
import yfinance as yf

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [2]:
stocks = pd.read_csv('ACOES.csv')
stocks["TICKET AJUSTADO"] = stocks['TICKET'] + '.SA'
stocks = stocks.sort_values(by='SETOR')


,SETOR,TICKET,STATUS,TICKET AJUSTADO
7,ALIMENTACAO,CAML3,FUNDAMENTOS OK,CAML3.SA
3,BANCARIO,BIDI4,FUNDAMENTOS OK,BIDI4.SA
5,BANCARIO,B3SA3,FUNDAMENTOS OK,B3SA3.SA
12,BANCARIO,ITSA4,FUNDAMENTOS OK,ITSA4.SA
2,COMUNICACAO,OIBR3,ESPECULACAO,OIBR3.SA


In [25]:
y_stocks = stocks[['TICKET AJUSTADO', 'TICKET']]

stock_data = yf.download(list(y_stocks['TICKET AJUSTADO']),  period = "ytd")
stock_closed_vl = stock_data['Close']

[*********************100%***********************]  20 of 20 completed


In [26]:
# Calculate expected returns and sample covariance
mu = expected_returns.mean_historical_return(stock_closed_vl)
S = risk_models.sample_cov(stock_closed_vl)

# Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.portfolio_performance(verbose=True)

Expected annual return: 2550.5%
Annual volatility: 70.2%
Sharpe Ratio: 36.31


(25.505419166948236, 0.7018338918177655, 36.31260824543601)

In [27]:
latest_prices = get_latest_prices(stock_closed_vl)
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=600)
allocation, leftover = da.lp_portfolio()
print(allocation)
print(leftover)

{'BIDI4.SA': 9, 'OIBR3.SA': 61}
1.0300188064575195
